# Conversational Task Agent

Taskbot that assist users in multi-step tasks, such as baking a 
birthday  cake  or  fixing  a  scratch  on  a  car  —  and  adapt  those  instructions  based  on  the 
resources  and  tools  available  to  the  customer.  If,  for  example,  a  customer  ran  out  of  an 
ingredient  halfway  through  a  recipe  or  didn’t  have  a  specific  tool  for  a  DIY  project,  the 
taskbot had to adjust the plan and suggest possible solutions.

## Phase 1 - (Task Retriever): Implementing a search index for recipes with OpenSearch

## Indexing

The Indexing.py module's purpose is to handle the index operations, might these be to calculate the embeddings and interact with the pickle files used to store the embeddings, to create the index and mappings or to parse and store the recipes from the base file to OpenSearch.

In [7]:
from modules.Indexing import Indexing

indexer = Indexing()

indexer.calculateAndStoreEmbeddings()
print("Embeddings stored and encoded")
indexer.createOpenSearchMappings()
print("Mappings created")
indexer.readAndStoreRecipesFromFile("./receitas.json")
print("Recipes stored")

Embeddings stored and encoded

Creating index
Mappings created
{'clip_embeddings': [-0.018773436546325684,
                     0.004890465177595615,
                     0.028203768655657768,
                     -0.029538070783019066,
                     -0.03738642856478691,
                     0.017758386209607124,
                     -0.019206546247005463,
                     -0.017766958102583885,
                     -0.022589439526200294,
                     -0.061956215649843216,
                     0.016700096428394318,
                     0.0054430607706308365,
                     0.023335637524724007,
                     -0.004184423945844173,
                     -0.012092088349163532,
                     0.027469269931316376,
                     0.014753039926290512,
                     0.02146572433412075,
                     0.035771314054727554,
                     -0.026913419365882874,
                     0.000518149696290493,
                     0.01

RequestError: RequestError(400, 'mapper_parsing_exception', "failed to parse field [clip_embeddings] of type [knn_vector] in document with id '0'. Preview of field's value: 'null'")

## Search

The Search.py module is simply an abstraction of the SearchBuilder.py module, made to simplify the search operations for a user. It's easily scalable thanks to the SearchBuilder layer and was made to be used as a simple interface for the system.

In [ ]:
from modules.Search import Search

search = Search()

# search.SearchTitleAndDescriptionTxt("chicken")
# search.SearchRecipeTime(30, 5)
# search.SearchRecipeNameTime(30, 5, "chicken")
# search.SearchRecipeIngredients("chicken curry")
# search.SearchRecipeExcludeIngredients("curry", "chicken")
# search.SearchRecipeDifficulty("easy")
# search.SearchRecipeNameDifficulty("easy", "chicken")
# search.SearchTitleEmbeddings("chicken")
# search.SearchDescriptionEmbeddings("chicken")

## SearchBuilder

The Search Builder API is meant to be used as a tool to easily and in an iterative manner build simple or complex search OpenSearch queries. This module avoids repetitive code, also as a way to ensure good application scaling and coding practices and maintaining a developer-friendly interface.

In [ ]:
import pprint as pp
from modules.SearchBuilder import SearchBuilder

searchBuilder = SearchBuilder()
searchBuilder.setSourceAsIdAndContent()
searchBuilder.setResultLength(5)
searchBuilder.setMandatoryTags(["Japanese"])
searchBuilder.setOptionalTags(["Sukiaki"])
searchBuilder.setIngredients("soy")
searchBuilder.excludeIngredients("chicken")
searchBuilder.setTimeFrame(150, 5)
result = searchBuilder.Search("beef Sukiyaki")
pp.pprint(result)

In [ ]:
# search for vegetarian pesto pasta
searchBuilder = SearchBuilder()
searchBuilder.setSourceAsIdAndContent()
searchBuilder.setResultLength(5)
searchBuilder.setMandatoryTags(['Vegan', 'Vegetarian'])
searchBuilder.setOptionalTags(['Gluten Free'])

response = searchBuilder.Search("Pesto pasta")
pp.pprint(response) # print

In [ ]:
# search for Yaki udon
searchBuilder = SearchBuilder()
searchBuilder.setSourceAsIdAndContent()
searchBuilder.setIngredients("mushrooms")
searchBuilder.excludeIngredients("brocolli")
searchBuilder.setOptionalTags(["Vegetarian"])
searchBuilder.setTimeFrame(20, 15)

response = searchBuilder.Search("Yaki udon with at home ingredients")
pp.pprint(response)# print

In [ ]:
# search for Dumpling Miso Soup Ramen
searchBuilder = SearchBuilder()
searchBuilder.setSourceAsIdAndContent()
searchBuilder.setIngredients("dumplings")
searchBuilder.setMandatoryTags(["Vegan", "Vegetarian", "Gluten Free"])
searchBuilder.setOptionalTags(["Main Dishes", "Japanese"])

response = searchBuilder.Search("Dumpling Miso Soup Ramen")
pp.pprint(response)# print

In [ ]:
# search for Ceasar Salad
searchBuilder = SearchBuilder()
searchBuilder.setSourceAsIdAndContent()
searchBuilder.setOptionalTags(["Main Dishes"])

response = searchBuilder.SearchByDescriptionEmbeddings("Ceasar Salad")
pp.pprint(response)# print

In [5]:
from opensearchpy import OpenSearch

host = 'api.novasearch.org'
port = 443
user = 'user212'
password = 'soO-2518'
index_name = user
client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = (user, password),
    url_prefix = 'opensearch',
    use_ssl = True,
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False
)

## Closing the client connection

In [ ]:
be sure you want to close the connection before running this code

resp = client.indices.close(index = index_name, timeout=600)
print(resp)

## Deleting the Index

In [6]:
 be sure you want to delete the index before running this code

if client.indices.exists(index=index_name):
    # Delete the index.
    response = client.indices.delete(
        index = index_name,
        timeout = 600
    )
    print('\nDeleting index:')
    print(response)


Deleting index:
{'acknowledged': True}
